In [1]:
import glob
import os
import numpy as np
import cv2
import imutils
import math
from scipy import misc
import copy

In [2]:
image = "data\\001_l_940_01.jpg"
original_img = cv2.imread(image)
head, tail = os.path.split(image)
name = tail.split('_')
print(name)

['001', 'l', '940', '01.jpg']


In [3]:
#rotate image
img_rotate = imutils.rotate_bound(original_img, 90)

#graylevel image
img_gray = cv2.cvtColor(img_rotate, cv2.COLOR_BGR2GRAY)

#otsu's thresholding
blur = cv2.GaussianBlur(img_gray,(5,5),0)
ret3,img_thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#morphological opening
sz = 20
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*sz-1, 2*sz-1))
opening = cv2.morphologyEx(img_thresh, cv2.MORPH_OPEN, kernel)

In [64]:
#first pixel with value 255 => vertical to horizontal

    #first column
column = np.argwhere(np.any(opening==255, axis = 0))[0]
c = column[0]

    #first row
row_columnX = opening[:,c:c+1]
row = np.argwhere(np.any(row_columnX==255, axis = 1))[0]
r = row[0]

In [65]:
ctrImg = np.zeros(opening.shape)
height, width = opening.shape
ctrImg[r,c] = 255
coor = [r,c]
startCoor = [r,c]

maksCoorR, maksCoorC = opening.shape
coorHalfWrist = [int(maksCoorR), int(maksCoorC/2)]
print(ctrImg.shape)

(768, 576)


In [66]:
newCoor = [0,0]
coorDis = []
direction = []
dire = 7
indikator = 1
count = 0
counts = 0
while indikator == 1:
    counts += 1
    if dire % 2 == 0:
        startDir = (dire+7) % 8
    else:
        startDir = (dire+6) % 8
    currentDir = startDir
    for i in range(8):
        if currentDir == 8:
            currentDir = 0
        if currentDir == 0:
            posisi = [0, 1]
        elif currentDir == 1:
            posisi = [-1, 1]
        elif currentDir == 2:
            posisi = [-1, 0]
        elif currentDir == 3:
            posisi = [-1, -1]
        elif currentDir == 4:
            posisi = [0, -1]
        elif currentDir == 5:
            posisi = [1, -1]
        elif currentDir == 6:
            posisi = [1, 0]
        elif currentDir == 7:
            posisi = [1, 1]
        dire = currentDir
        newCoor[0] = coor[0] + posisi[0]
        newCoor[1] = coor[1] + posisi[1]
        if newCoor[0] >= 0 and newCoor[1] >= 0 and newCoor[0] < maksCoorR-1 and newCoor[1] < maksCoorC-1:
            if opening[newCoor[0], newCoor[1]] == 255:
                ctrImg[newCoor[0], newCoor[1]] = 255
                coor[0] = newCoor[0]
                coor[1] = newCoor[1]
                a = math.pow((newCoor[0] - coorHalfWrist[0] + 1),2)
                b = math.pow((newCoor[1] - coorHalfWrist[1] + 1),2)
                c = a + b
                distances = math.sqrt(c)
                coorDis.append([])
                direction.append([])
                coorDis[count].append(newCoor[0])
                coorDis[count].append(newCoor[1])
                coorDis[count].append(distances)
                direction[count].append(dire)
                count += 1
                break
            else:
                currentDir += 1
        else:
            currentDir += 1
    if startCoor == coor:
        indikator = 0

In [67]:
sumbuX = []
sumbuX = np.arange(len(coorDis))
sumbuX = np.array(sumbuX)

# minumum Distance 
distances = []
for i in range(len(coorDis)):
    distances.append(coorDis[i][2])

minDistance = np.amin(distances)
checkImage = copy.deepcopy(ctrImg)
print(minDistance)

#index pixel with minimum distance
indexDistance = np.argwhere(coorDis == minDistance)
    # 2 == distance
for i in indexDistance:
    if i[1] == 2:
        r = i[0]
        break
c = 0
x, y, d = coorDis[r]

    #cek start
for j in range(100):
    checkImage[x][y+j] = 255

43.0116263352


In [68]:
# sequence 
newCoorDis = []
for i in range(len(coorDis) - r):
    newCoorDis.append(coorDis[i+r])
for i in range(r):
    newCoorDis.append(coorDis[i])
newCoorDis = np.array(newCoorDis)

height, width = ctrImg.shape
image = copy.deepcopy(ctrImg)

    #checking
new = newCoorDis[0]
for i in range(150):
    image[int(new[0])][int(new[1]) + i] = 255
    

In [69]:
whiteDis = float(570)
r = []
c = []
for i in range(len(newCoorDis)):
    if newCoorDis[i][0] > whiteDis:
        newCoorDis[i][2] = 0
        
distancess = []
for i in range(len(newCoorDis)):
    distancess.append(newCoorDis[i][2])
distancess = np.array(distancess)

In [70]:
def getPeak(Data, i, s):
    if (i-s) < 1:
        top = 0
    else:
        top = i - s - 1
    y = Data[1]
    
    if (i+s) > len(y):
        bottom = len(y) - 1
    else:
        bottom = i + s - 1
    p = []
    if y[i] == max(y[top:bottom]) or y[i] == min(y[top:bottom]):
        p = [Data[0][i], Data[1][i]]
    else:
        p.append(float('nan'))
        p.append(float('nan'))
    return p

In [71]:
def fpeak(sumbuX, distancess, s):
    rx = len(sumbuX)
    ry = len(distancess)
    if s == 0:
        s = 1
    Data = np.vstack((sumbuX, distancess))
    output = []
    for i in range(rx):
        isP= getPeak(Data, i, s)
        if np.isnan(isP).sum() == 0:
            output.append(isP)
    return output

In [72]:
s = 50
output = fpeak(sumbuX, distancess, s)

for i in range(len(output)):
    if i == len(output) - 1:
        output[i].append(0)
        break
    if output[i][1] == output[i+1][1]:
        output[i].append(1)
    else:
        output[i].append(0)

improveOut = []
for i in output:
    if i[2] == float(0):
        improveOut.append(i)
# sorted(improveOut, reverse=True)
output = improveOut

In [ ]:
pixelP1 = output[0][0]
pixelP2 = output[6][0]
coorP1 = [newCoorDis[int(pixelP1)][0], newCoorDis[int(pixelP1)][1]]
coorP2 = [newCoorDis[int(pixelP2)][0], newCoorDis[int(pixelP2)][1]]

# kemiringan p1 dam p2
temp = (coorP2[1] - coorP1[1]) / (coorP2[0] - coorP1[0])
deg = math.atan(temp) / math.pi * 180

newImg = misc.imrotate(img_rotate,90 - deg,interp='bilinear')

temp = np.zeros_like(img)
height, width = img.shape
temp[int(coorP1[0])][int(coorP1[1])] = 255
temp[int(coorP2[0])][int(coorP2[1])] = 255

tempRotate = misc.imrotate(temp,90 - deg,interp='bilinear')

tempIndex = np.argwhere(tempRotate>0)

newCoorP1 = [tempIndex[0][0], tempIndex[0][1]]
newCoorP2 = [tempIndex[len(tempIndex)-1][0], tempIndex[len(tempIndex)-1][1]]

a = math.pow((newCoorP1[0]-newCoorP2[0]),2)
b = math.pow((newCoorP1[1]-newCoorP2[1]),2)
distance = math.sqrt(a+b)
distance = int(distance)
image = copy.deepcopy(newImg)
#     ROI output
fiturI = newImg[int(newCoorP1[0]):int(newCoorP1[0]+distance), int(newCoorP1[1]):int(newCoorP1[1]+distance)]
for a in range(distance):
    for b in range(distance):
        if (a+newCoorP1[0]) < height and (b+newCoorP1[1]) < width:
            image[a+newCoorP1[0]][b+newCoorP1[1]] = 255

In [10]:
img = [[0,0,0,0,0],
      [0,0,1,1,0],
      [0,1,1,1,0],
      [0,1,1,1,1],
      [0,0,1,1,1],
      [0,0,0,0,0]]
img = np.array(img)
column = np.argwhere(np.any(img==1, axis = 0))
c = int(column[0])
print(img)

row_columnX = img[:,c:c+1]
print(row_columnX)
row = np.argwhere(np.any(row_columnX==1, axis = 1))[0]
r = row[0]
print(r)

[[0 0 0 0 0]
 [0 0 1 1 0]
 [0 1 1 1 0]
 [0 1 1 1 1]
 [0 0 1 1 1]
 [0 0 0 0 0]]
[[0]
 [0]
 [1]
 [1]
 [0]
 [0]]
2
